# Training cycle debugging

Basically run code copied from train_seq2seq.py in this notebook to catch bugs

In [2]:
import os
os.environ['ALFRED_ROOT'] = '/home/hoyeung/alfred/'

import sys
sys.path.append(os.path.join(os.environ['ALFRED_ROOT']))
sys.path.append(os.path.join(os.environ['ALFRED_ROOT'], 'models'))

# from argparse import ArgumentDefaultsHelpFormatter, ArgumentParser
import torch
import pprint
import json
from data.preprocess import Dataset
from importlib import import_module, reload
from argparse import ArgumentDefaultsHelpFormatter, ArgumentParser
from models.utils.helper_utils import optimizer_to

In [3]:
# import these if we want to debug a model such as seq2seq_nl_baseline.py

import torch
import pprint
import json
from data.preprocess import Dataset
from importlib import import_module, reload
from argparse import ArgumentDefaultsHelpFormatter, ArgumentParser
from models.utils.helper_utils import optimizer_to

# Parser

In [4]:
# These are the default flags present in train_seq2seq.py

parser = ArgumentParser(formatter_class=ArgumentDefaultsHelpFormatter)
args = parser.parse_args('')

# settings
args.seed = 123
args.data = 'data/json_feat_2.1.0'
args.splits = 'data/splits/oct21.json'
args.preprocess = False #!
args.pp_folder = 'pp'
args.save_every_epoch = False #!
args.model = 'seq2seq_nl_baseline'
args.gpu = True
args.dout = 'exp/model:seq2seq_nl_baseline'
args.resume = False #!

# hyper parameters
args.batch = 8
args.epoch = 20
args.lr = 1e-4
args.decay_epoch = 10
args.dhid = 512
args.dframe = 2500
args.demb = 100
args.pframe = 300
args.mask_loss_wt = 1.
args.action_loss_wt = 1.
args.subgoal_aux_loss_wt = 0.
args.pm_aux_loss_wt = 0.

# dropouts
args.zero_goal = False #!
args.zero_instr = False #!
args.act_dropout = 0.
args.lang_dropout = 0.
args.input_dropout = 0.
args.vis_dropout = 0.3
args.hstate_dropout = 0.3
args.attn_dropout = 0.
args.actor_dropout = 0.
args.word_dropout = 0.

# other settings
args.dec_teacher_forcing = False #!
args.temp_no_history = False #!

# debugging
args.fast_epoch = False #!
args.dataset_fraction = 0

In [11]:
# overwrite the default flags

args.preprocess = False # Turn this to True if running for the first time

args.model = 'seq2seq_nl_baseline'  # found under models/model/ directory
args.dout = 'exp/model:seq2seq_nl_baseline'

args.dec_teacher_forcing = True
# args.gpu = False

# light setup for debugging
args.fast_epoch = True # Turn this to False if running for the first time to preprocess data properly
args.epoch = 2

In [6]:
torch.manual_seed(args.seed)

# Setup and load data splits

In [7]:
# make output dir
if not os.path.isdir(args.dout):
    os.makedirs(args.dout)

print('args.out ', args.dout)

args.out  exp/model:seq2seq_nl_baseline


In [8]:
# load train/valid/tests splits
with open(args.splits) as f:
    splits = json.load(f)
    pprint.pprint({k: len(v) for k, v in splits.items()})

{'tests_seen': 1533,
 'tests_unseen': 1529,
 'train': 21023,
 'valid_seen': 820,
 'valid_unseen': 821}


In [9]:
# preprocess and save -- only need to preprocess once
if args.preprocess:
    print("\nPreprocessing dataset and saving to %s folders ... This will take a while. Do this once as required." % args.pp_folder)
    dataset = Dataset(args, None)
    dataset.preprocess_splits(splits)
    vocab = torch.load(os.path.join(args.dout, "%s.vocab" % args.pp_folder))
else:
    vocab = torch.load(os.path.join(args.data, "%s.vocab" % args.pp_folder))
    
print(vocab)

{'word': Vocab(2360), 'action_low': Vocab(15), 'action_high': Vocab(93)}


# Model and Training

In [24]:
# load the model architecture
args.gpu = False

M = import_module('model.{}'.format(args.model))
reload(M)
model = M.Module(args, vocab)
optimizer = None

if args.gpu:
    model = model.to(torch.device('cuda'))
    if not optimizer is None:
        optimizer_to(optimizer, torch.device('cuda'))

In [25]:
# examine model layers

model

Module(
  (emb_word): Embedding(2360, 100)
  (emb_action_low): Embedding(15, 100)
  (enc): LSTM(100, 512, batch_first=True, bidirectional=True)
  (enc_att): SelfAttn(
    (scorer): Linear(in_features=1024, out_features=1, bias=True)
  )
  (dec): LanguageDecoder(
    (emb): Embedding(2360, 100)
    (cell): LSTMCell(1124, 1024)
    (attn): DotAttn()
    (input_dropout): Dropout(p=0.0, inplace=False)
    (attn_dropout): Dropout(p=0.0, inplace=False)
    (hstate_dropout): Dropout(p=0.3, inplace=False)
    (word_dropout): Dropout(p=0.0, inplace=False)
    (word): Linear(in_features=2148, out_features=100, bias=True)
    (h_tm1_fc): Linear(in_features=1024, out_features=1024, bias=True)
  )
  (act_dropout): Dropout(p=0.0, inplace=True)
)

In [26]:
# main training loop -- debug here if breakpoints were inserted
model.run_train(splits, optimizer=optimizer)

batch:   0%|          | 0/2 [00:00<?, ?it/s]

Saving to: exp/model:seq2seq_nl_baseline



batch: 100%|██████████| 2/2 [00:09<00:00,  4.93s/it]

batch: 100%|██████████| 2/2 [00:05<00:00,  2.66s/it]

batch: 100%|██████████| 2/2 [00:05<00:00,  2.54s/it]



Found new best valid_seen!! Saving...
Found new best valid_unseen!! Saving...


batch:   0%|          | 0/2 [00:00<?, ?it/s]

{'epoch': 0,
 'train': {'lang_instr_bleu': 5.444214570154877e-232,
           'loss_lang_instr': 5.235116481781006,
           'total_loss': 5.235116481781006},
 'valid_seen': {'lang_instr_bleu': 1.1392305462353929e-156,
                'loss_lang_instr': 4.714325189590454,
                'total_loss': 4.714325189590454},
 'valid_unseen': {'lang_instr_bleu': 5.048127627446426e-232,
                  'loss_lang_instr': 4.025876998901367,
                  'total_loss': 4.025876998901367}}
epoch_time			28.524
forward_batch_train			9.866
forward_batch_valid_seen			5.334
forward_batch_valid_unseen			5.094
compute_metrics_valid_unseen			4.352
compute_metrics_valid_seen			2.863
compute_metrics_train			0.403
make_debug_train			0.115
make_debug_valid_seen			0.112
torch_save_valid_unseen			0.112
torch_save_valid_seen			0.111
torch_save_last			0.108
make_debug_valid_unseen			0.051
setup_time			0.002



batch: 100%|██████████| 2/2 [00:10<00:00,  5.48s/it]

batch: 100%|██████████| 2/2 [00:03<00:00,  1.64s/it]

batch: 100%|██████████| 2/2 [00:03<00:00,  1.57s/it]



Found new best valid_seen!! Saving...
Found new best valid_unseen!! Saving...


epoch: 100%|██████████| 2/2 [00:48<00:00, 24.14s/it]

{'epoch': 1,
 'train': {'lang_instr_bleu': 7.760647204734706e-232,
           'loss_lang_instr': 4.80204176902771,
           'total_loss': 4.80204176902771},
 'valid_seen': {'lang_instr_bleu': 9.296291086808552e-232,
                'loss_lang_instr': 4.26695191860199,
                'total_loss': 4.26695191860199},
 'valid_unseen': {'lang_instr_bleu': 7.449444395162465e-232,
                  'loss_lang_instr': 3.7604312896728516,
                  'total_loss': 3.7604312896728516}}
epoch_time			48.272
forward_batch_train			20.829
forward_batch_valid_seen			8.669
forward_batch_valid_unseen			8.248
compute_metrics_valid_unseen			4.696
compute_metrics_valid_seen			3.231
compute_metrics_train			0.811
torch_save_valid_seen			0.765
torch_save_valid_unseen			0.261
torch_save_last			0.257
make_debug_valid_unseen			0.168
make_debug_train			0.166
make_debug_valid_seen			0.161
setup_time			0.002


# Look at debugging outputs

In [51]:
!ls -l exp/model:seq2seq_nl_baseline

total 451936
-rw-r--r-- 1 root root       453 Apr  8 01:23 best_seen.json
-rw-r--r-- 1 root root 153981936 Apr  8 01:23 best_seen.pth
-rw-r--r-- 1 root root       453 Apr  8 01:23 best_unseen.json
-rw-r--r-- 1 root root 153981936 Apr  8 01:23 best_unseen.pth
-rw-r--r-- 1 root root       843 Apr  8 01:22 config.json
-rw-r--r-- 1 root root       250 Apr  7 23:56 events.out.tfevents.1586303655.3c556c990178
-rw-r--r-- 1 root root       250 Apr  8 00:25 events.out.tfevents.1586305388.3c556c990178
-rw-r--r-- 1 root root         0 Apr  8 00:24 events.out.tfevents.1586305447.3c556c990178
-rw-r--r-- 1 root root       250 Apr  8 00:28 events.out.tfevents.1586305598.3c556c990178
-rw-r--r-- 1 root root       250 Apr  8 00:37 events.out.tfevents.1586306114.3c556c990178
-rw-r--r-- 1 root root      7108 Apr  8 01:14 events.out.tfevents.1586308348.3c556c990178
-rw-r--r-- 1 root root      4095 Apr  8 01:23 events.out.tfevents.1586308974.3c556c990178
-rw-r--r-- 1 root root 153981936 Apr  8 01:23 latest.

In [5]:
epoch = 4

with open(os.path.join(args.dout, 'valid_unseen.debug_epoch_{}.preds.json'.format(epoch)), 'r') as f:
    debug_tr = json.load(f)

In [6]:
print(type(debug_tr))
print(len(debug_tr))

<class 'dict'>
16


In [10]:
# all the task numbers
print(debug_tr.keys())
# let's look at one of them below
task_num = 'trial_T20190907_155006_533262_0'

dict_keys(['trial_T20190908_044113_026049_0', 'trial_T20190907_020258_749030_0', 'trial_T20190909_061130_844814_0', 'trial_T20190907_155006_533262_0', 'trial_T20190906_191501_563086_0', 'trial_T20190912_221141_608117_0', 'trial_T20190909_061838_159982_0', 'trial_T20190908_145356_918528_0', 'trial_T20190907_171035_866841_0', 'trial_T20190907_161235_786153_0', 'trial_T20190907_133953_562557_0', 'trial_T20190907_171933_349922_0', 'trial_T20190908_032518_891433_0', 'trial_T20190909_120632_691361_0', 'trial_T20190907_221208_560499_0', 'trial_T20190908_073749_086690_0'])


In [64]:
# We log these from each task
debug_tr[task_num].keys()

dict_keys(['lang_goal', 'lang_instr', 'action_low', 'action_high', 'p_lang_instr'])

In [11]:
# Gold goal description
debug_tr[task_num]['lang_goal']

'Put a washed bowl away in a kitchen cabinet.'

In [12]:
# Gold low-level instruction description
debug_tr[task_num]['lang_instr']

['Turn left, then go clockwise to face the opposite end of the kitchen island.',
 'Pick up the bowl that is near the potted plant in the corner of the island.',
 'Turn right and go counterclockwise around the island to the sink.',
 'Wash the bowl in the sink.',
 'Face the opposite direction and look up at the cabinets above the counter to the left of the stove.',
 'Put the bowl on the bottom shelf of the cabinet on the right.']

In [67]:
# Gold Input low-level action description
debug_tr[task_num]['action_low']

['LookDown_15',
 'RotateLeft_90',
 'MoveAhead_25',
 'MoveAhead_25',
 'RotateRight_90',
 'MoveAhead_25',
 'MoveAhead_25',
 'MoveAhead_25',
 'MoveAhead_25',
 'MoveAhead_25',
 'MoveAhead_25',
 'RotateRight_90',
 'LookDown_15',
 'PickupObject',
 'LookUp_15',
 'RotateLeft_90',
 'MoveAhead_25',
 'MoveAhead_25',
 'RotateLeft_90',
 'MoveAhead_25',
 'LookUp_15',
 'LookUp_15',
 'LookUp_15',
 'OpenObject',
 'PutObject',
 'CloseObject',
 'ToggleObjectOn',
 'ToggleObjectOff',
 'OpenObject',
 'PickupObject',
 'CloseObject',
 'LookDown_15',
 'LookDown_15',
 'LookDown_15',
 'RotateLeft_90',
 'MoveAhead_25',
 'MoveAhead_25',
 'RotateLeft_90',
 'MoveAhead_25',
 'PutObject']

In [68]:
# Gold Input high-level subgoal action description
debug_tr[task_num]['action_high']

['GotoLocation',
 'PickupObject',
 'GotoLocation',
 'HeatObject',
 'GotoLocation',
 'PutObject',
 'NoOp']

In [69]:
# Prediction result
debug_tr[task_num]['p_lang_instr']

'the cater faces tilt combo corner hearth let pick mixing running freezer . vase bathrub organizer flattest reuse spring the hutch flattest spaces organize study coach puts bringing tiny passed sprayers toilets the hutch coffeemaker the hutch coffeemaker the hutch coffeemaker the hutch coffeemaker the hutch coffeemaker the hutch coffeemaker the hutch coffeemaker the hutch coffeemaker the hutch coffeemaker the hutch coffeemaker the hutch coffeemaker the hutch coffeemaker the hutch coffeemaker the hutch coffeemaker the hutch coffeemaker the hutch coffeemaker the hutch coffeemaker the hutch coffeemaker the hutch coffeemaker the hutch coffeemaker the hutch coffeemaker the hutch coffeemaker the hutch coffeemaker the hutch coffeemaker the hutch coffeemaker the hutch coffeemaker the hutch coffeemaker the hutch coffeemaker the hutch coffeemaker the hutch coffeemaker the hutch coffeemaker the hutch coffeemaker the hutch coffeemaker the hutch coffeemaker the hutch coffeemaker the hutch coffeemak